In [1]:
import os

requirements_installed = False
max_retries = 3
retries = 0


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return

In [ ]:
install_requirements()

In [3]:
from dotenv import load_dotenv
import os


def setup_env():
    """Sets up the environment variables"""
    load_dotenv()

    GROQ_API_KEY = os.getenv("GROQ_API_KEY")

    if GROQ_API_KEY is None:
        print("Please set the GROQ_API_KEY environment variable.")
        exit(1)
    else:
        print("GROQ_API_KEY is set.")

In [ ]:
setup_env()

In [5]:
import instructor
from groq import Groq
import traceback
from pydantic import BaseModel
from typing import Union

DEFAULT_MODEL = "llama-3.3-70b-versatile"


class LLMErrorResponse(BaseModel):
    error: str


def get_groq_client():
    """Returns an instance of the Groq class"""
    groq = Groq(api_key=os.getenv("GROQ_API_KEY"))
    client = instructor.from_groq(groq, mode=instructor.Mode.JSON)
    return client


def llm(
    prompt: str,
    response_model: BaseModel,
    system="You are a helpful AI assistant. The user will talk to you and its your job to provide detailed and clear responses.",
    model=DEFAULT_MODEL,
) -> Union[BaseModel, LLMErrorResponse]:
    """Calls LLM API with the given prompt. Defaults to llama-3.3-70b-versatile""",
    try:
        client = get_groq_client()

        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ]

        response = client.chat.completions.create(
            messages=messages, model=model, response_model=response_model
        )
        return response
    except Exception as e:
        traceback.print_exc()
        return LLMErrorResponse(error=str(e))

In [6]:
from typing import List, Union, Any
from pydantic import BaseModel
from groq import Groq


class UserStory(BaseModel):
    """Represents a User Story"""

    title: str
    description: str
    acceptance_criteria: List[str]
    story_points: int


class UserStories(BaseModel):
    """Represents a collection of User Stories"""

    user_stories: List[UserStory]


class PRD(BaseModel):
    """Represents a Product Requirements Document (PRD)"""

    title: str
    overview: str
    problem_statement: str
    business_domains: List[str]
    proposed_solution: str
    target_audience: str
    features: List[str]
    constraints: List[str]
    assumptions: List[str]
    dependencies: List[str]
    user_personas: List[str]
    conclusion: str


class MiniPM:
    """An AI agent that acts as a Product Manager to help project managers create PRD and User Stories for their projects."""

    llm: Groq
    project_description: str

    def __init__(self, project_description: str):
        """Initializes the MiniPM AI agent with the project description."""
        self.project_description = project_description
        self.llm = get_groq_client()

    def get_prd(
        self, response_format="markdown", augment=True
    ) -> Union[PRD, LLMErrorResponse, Any]:
        """Gets the Product Requirements Document (PRD) for the project."""
        system = """
                You are MiniPM; a Product Manager AI agent. 
                You help project managers to create a Product Requirements Document (PRD) for their projects.
                """
        prompt = f"""
                Provide a Product Requirements Document (PRD) for the project with the following details:
                Details: {self.project_description}
                """
        response = llm(prompt=prompt, response_model=PRD, system=system)

        if augment:
            response = self._augment_prd(response)

        if response_format == "json":
            return response.model_dump_json()
        elif response_format == "markdown":
            response_dict = response.model_dump()
            keys = response_dict.keys()
            markdown = f"# Product Requirements Document (PRD) for {response.title}\n"
            for key in keys:
                title = key.title().replace("_", " ").replace("Prd", "PRD")
                if key == "title":
                    title = ""
                    continue
                markdown += f"## {title}\n"
                value = response_dict[key]
                if type(value) == list:
                    value_bullets = [f"- {item}" for item in value]
                    value = "\n".join(value_bullets)
                    markdown += f"{value}\n"
                else:
                    markdown += f"{response_dict[key]}\n"
            return markdown
        elif response_format == "dto":
            return response
        raise ValueError("Invalid response_format. Use 'json', 'markdown' or 'dto'.")

    def _augment_prd(self, prd: PRD) -> PRD:
        """Augments the PRD with greater details."""
        prd_json = prd.model_dump()
        print("Augmenting PRD: ", prd_json)
        keys = prd_json.keys()
        for key in keys:
            if key == "title":
                continue
            value = prd_json[key]
            prompt = f"Provide more details about the {key} for the project."
            if type(value) == str:
                prompt += f" Current value: {value}"
                expanded_value = llm(prompt, response_model=str)
                prd_json[key] = expanded_value
            elif type(value) == list:
                expanded_values = []
                for item in value:
                    prompt += f" Current value: {item}"
                    expanded_item = llm(prompt, response_model=str)
                    expanded_values.append(expanded_item)
                prd_json[key] = expanded_values
        print("Augmented PRD: ", prd_json)
        return PRD(**prd_json)

    def get_user_stories(self) -> Union[UserStories, LLMErrorResponse]:
        """Gets the User Stories for the project"""
        system = """
                You are MiniPM; a Product Manager AI agent. 
                You help project managers to create User Stories for their projects.
                """
        prompt = f"""
                Provide User Stories for the project with the following details:
                Details: {self.project_description}
                PRD: {self.get_prd()}
                """
        response = llm(prompt=prompt, response_model=UserStories, system=system)
        return response

In [7]:
from IPython.display import Markdown


def render_output(markdown: str) -> None:
    """Renders the generated output file as markdown."""
    return Markdown(markdown)


def save_markdown_file(markdown: str, file_path: str) -> None:
    """Saves the generated markdown file to the specified path."""
    with open(file_path, "w") as file:
        file.write(markdown)

In [ ]:
## Input Parameters: Change these values to get different PRD and User Stories
project_description = (
    "A web application for tracking investigating behavioural change in aliens."
)
output_file = "alien_behavioural_change_prd.md"
augment = True

## Create an instance of MiniPM AI agent
minipm = MiniPM(project_description)

## Get the Product Requirements Document (PRD)
prd = minipm.get_prd(response_format="markdown", augment=augment)

save_markdown_file(prd, output_file)

## Render the PRD as markdown
render_output(prd)

In [ ]:
## Input Parameters: Change these values to get different PRD and User Stories
project_description = "A web application for tracking fitness goals and workouts."

## Create an instance of MiniPM AI agent
minipm = MiniPM(project_description)

## Get the Product Requirements Document (PRD)
user_stories = minipm.get_user_stories()

## Render the PRD as markdown
print(user_stories.model_dump_json(indent=4))